<h1><center> The Cooley-Tukey radix-2 FFT in Fortran </center></h1>

## 1. Background

### 1.1 One-dimensional case

The Fourier Transform on a discrete set of points is defined as:

$\displaystyle \tilde{x}[k] = \sum_{n=0}^{N-1} x[n] \, e^{-2 \pi i kn/N} \tag{1}$

This is called the *Discrete Fourier Transform* (DFT). The input array $x[n]$ and the output array $\tilde{x}[k]$ have the same number of elements - $N$. If computed as per the formula above, the DFT requires $N^2$ operations, which can make it very slow on large enough arrays. 

One way to reduce the number of calculations we need to do is to exploit a symmetry in the formula. Let's split the sum into an even and into an odd part:

$\displaystyle \tilde{x}[k] = \sum_{m=0}^{N/2-1} x[2m] \, e^{-2 \pi i k (2m)/N} + \sum_{m=0}^{N/2-1} x[2m+1] \, e^{-2 \pi i k (2m+1)/N},  \tag*{}$

$\displaystyle \tilde{x}[k] = \sum_{m=0}^{N/2-1} x[2m] \, e^{-2 \pi i k m/(N/2)} + e^{-2 \pi i k/N} \sum_{m=0}^{N/2-1} x[2m+1] \, e^{-2 \pi i k m/(N/2)},  \tag*{}$

$\displaystyle \tilde{x}[k] = \text{DFT}(x_\text{even})[k] + e^{-2 \pi i k/N} \, \text{DFT}(x_\text{odd})[k]. \tag{2}$

Therefore we can see that we've split the original $N^2$ problem into two $(N/2)^2$ problems and hence we are now $2 \times$ faster. To add up the two results we need another $N$ steps, so that the total is $N^2/2 + N$. Note that there is a slight abuse of notation here. The DFT's on the even and odd elements produce arrays which are $N/2$ long while $k$ runs from $0$ to $N-1$. This fixed by setting $\text{DFT}(x_\text{even})[k+N/2] = \text{DFT}(x_\text{even})[k]$ (similarly for the odd elements).

The key realisation is that if the input array has length which is a power of two ($N=2^p$), we can keep splitting the resultant DFTs into even and odd parts all the way down. Then we will have $N^2/2^p + p \times N$ operations to do. Thus the complexity now is $\mathcal{O}(N \log_2 N)$. This is the radix-2 Cooley-Tukey algorithm used to compute the Fast Fourier Transform (FFT).

### 1.2 Two-dimensional case

In two dimensions, the Discrete Fourier Transform is defined as:

$\displaystyle \tilde{x}[u,v] = \sum_{n=0}^{N-1} \sum_{m=0}^{N-1} x[n,m] \, e^{-2 \pi i (un + vm)/N}. \tag{3}$

We can make use of the 1D FFT from above to speed this up as well. To do that, notice the 2D DFT can be calculated by performing individual 1D FFTs:

$\displaystyle \tilde{x}[u,v] = \sum_{n=0}^{N-1} e^{-2 \pi i un/N} \sum_{m=0}^{N-1} x[n,m] \, e^{-2 \pi i vm/N}. \tag{4}$

The process is essentially:
 - Create a matrix whose rows are FFTs of the rows of $x$,
 - Replace each column in that matrix by its FFT.

## 2. Implementation

### 2.1 Setting everything up

For maximal computational efficiency, I let Fortran subroutines to do the big number crunching. To do this, the subroutines are packaged up into a python module using `numpy.f2py`. The fortran file `fortranfft.f95` contains a module called `fortran_subroutines` which has all the functions we need for the implementation. This code is compiled and turned into a shared object file `.so` which can be called from python. It is created with the command:

`python -m numpy.f2py -c fortranfft.f95 -m fmodule`

To import it into Python we just add `import fmodule`. 

In order to facilitate usability and to perform various checks on the inputs, we hide everything in the final python module `fortranfft`.

### 2.2 Efficiency

I have tried to vectorise the code to a decent degree, and also tried to be a bit smart with memory management. The calculations require the excessive use of the array $W[i] = \exp(-2i \pi n/N)$ so these are calculated only once when `fortranfft.fft(x)` is called and are passed down on each iteration as opposed to recalculated. 

A good number of the calculations are performed using the Single Instruction Multiple Data (SIMD) registers in x86 processors (labeled `%xmm`). This can be observed by compiling and checking the assembly code of the fortran module. We can do that by adding the compiler flag `-S`: 

`gfortran -S fortranfft.f95`

A significant downside is that the fortran subroutines make use of only one core. This could potentially be fixed with APIs such as `OpenMP`.

### 2.3 Comparison with `numpy`

We compare the execution times of the fortran module with those of the FFT functions contained in `numpy`. Numpy's core functions are written in C as opposed to Fortran.

In [49]:
import numpy as np
import fortranfft as f
import timeit as t

x = np.random.random(2048)

t1 = t.default_timer()
f1 = f.fft(x)
t2 = t.default_timer()

t3 = t.default_timer()
f2 = np.fft.fft(x)
t4 = t.default_timer()

print('Result is the same as numpy: %r' %(np.allclose(f1,f2)))
print('Fortran: %e sec' %(t2-t1))
print('Numpy  : %e sec' %(t4-t3))

Result is the same as numpy: True
Fortran: 8.006500e-04 sec
Numpy  : 3.172210e-04 sec


As you can see Numpy is a few times faster still. 

And the 2D FFT:

In [51]:
x = np.random.rand(2048,2048)

t1 = t.default_timer()
f1 = f.fft2(x)
t2 = t.default_timer()

t3 = t.default_timer()
f2 = np.fft.fft2(x)
t4 = t.default_timer()

print('Result is the same as numpy: %r' %(np.allclose(f1,f2)))
print('Fortran: %e sec' %(t2-t1))
print('Numpy  : %e sec' %(t4-t3))

Result is the same as numpy: True
Fortran: 1.334623e+00 sec
Numpy  : 1.990195e-01 sec


Much slower but not too bad for a rough implementation.